<a href="https://colab.research.google.com/github/hamm0o0d/Game-Rate-Prediction/blob/main/tst_script_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import libraries, load data and encode rate

In [207]:
pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [208]:
import numpy as np # for numerical operations
import pandas as pd # for handling input data
import joblib
import requests
import pickle

from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, precision_score, recall_score, f1_score, confusion_matrix 
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest,mutual_info_regression,f_classif
from sklearn.model_selection import GridSearchCV,KFold,train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from datetime import datetime
import category_encoders as ce
import re
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize 
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords,wordnet

In [209]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [210]:
!pip show pandas

Name: pandas
Version: 1.5.3
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, python-dateutil, pytz
Required-by: altair, arviz, category-encoders, cmdstanpy, cufflinks, datascience, db-dtypes, dopamine-rl, fastai, google-colab, gspread-dataframe, holoviews, mizani, mlxtend, pandas-datareader, pandas-gbq, plotnine, prophet, pymc, seaborn, sklearn-pandas, statsmodels, vega-datasets, xarray, yfinance


# load data

In [211]:
data=pd.read_csv('/content/ms2-games-tas-test-v2.csv')
# va=pd.read_csv('/content/visual_appeal_score.csv')


# drop duplicates
data.drop_duplicates(inplace=True)
# data['visual_appeal']=va.values
print(data.shape)
data

(653, 18)


,URL,ID,Name,Subtitle,Icon URL,User Rating Count,Price,In-app Purchases,Description,Developer,Age Rating,Languages,Size,Primary Genre,Genres,Original Release Date,Current Version Release Date,Rate
0,https://apps.apple.com/us/app/my-colony/id1123...,1123806108,My Colony,NaN,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,682.0,0.00,4.99,My Colony is a brand new space colonization si...,Brandon Stecklein,4+,EN,131994624.0,Games,"Games, Simulation, Strategy",18/10/2016,2/7/2019,Intermediate
1,https://apps.apple.com/us/app/hidden-object-co...,893444350,Hidden Object Countryside Love,Casual puzzles full of fun!,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,19.0,0.00,"0.99, 1.99, 4.99, 9.99",Ready to start a journey full of fun and surpr...,Zheng Hui,4+,"EN, FR, DE, JA, RU, ZH, ES, ZH",47676416.0,Games,"Games, Strategy, Puzzle, Entertainment",17/10/2014,15/12/2018,Intermediate
2,https://apps.apple.com/us/app/haunted-towns-hi...,1122711408,Haunted Towns Hidden Object \u2013 Secret Myst...,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,9.0,0.99,NaN,Hidden Objects \u2013 Haunted Towns is an arti...,Detention Apps,4+,EN,65063936.0,Games,"Games, Strategy, Puzzle, Entertainment",10/6/2016,10/6/2016,Intermediate
3,https://apps.apple.com/us/app/battlepillars-mu...,664303561,Battlepillars: Multiplayer (PVP) Real Time Str...,NaN,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,1155.0,0.00,"0.99, 0.99, 1.99, 1.99, 4.99, 4.99, 9.99, 2.99...",Crawl into the bug battle with this action pac...,"Hitcents.com, Inc.",9+,EN,113438720.0,Games,"Games, Entertainment, Strategy, Action",2/10/2013,24/09/2015,High
4,https://apps.apple.com/us/app/chess-genius/id2...,294631172,Chess Genius,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,156.0,5.99,NaN,"""ChessGenius, award winning chess program. Sup...",Lang Software Limited,4+,"EN, FR, DE, IT, ES",1751040.0,Games,"Games, Strategy, Board, Entertainment",8/11/2008,5/8/2016,Intermediate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,https://apps.apple.com/us/app/antiquitas/id131...,1310689005,Antiquitas,Build your own ancient city!,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,26.0,0.00,2.99,"""Antiquitas is an ancient city builder and sim...",Brandon Stecklein,12+,EN,145886208.0,Games,"Games, Strategy, Simulation",17/11/2017,8/7/2019,Intermediate
650,https://apps.apple.com/us/app/simple-chess-boa...,531158560,Simple chess board,NaN,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,26.0,0.00,0.99,"""A simple chess board and a chess clock withou...",Luis Lopez,4+,"CA, EN, ES",7204864.0,Games,"Games, Strategy, Board, Utilities",5/7/2012,10/11/2015,High
651,https://apps.apple.com/us/app/wheres-my-jewel/...,689905568,"""Where's My Jewel?""",NaN,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,10.0,0.00,2.99,Download this fun jewel matching puzzle game.\...,Makeover Mania Story Games,4+,EN,12992512.0,Games,"Games, Strategy, Puzzle",24/08/2013,24/08/2013,Intermediate
652,https://apps.apple.com/us/app/city-island-2-bu...,838193156,City Island 2: Building Story,Fun Offline City Building Game,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,652.0,0.00,"2.99, 2.99, 7.99, 2.99, 1.99, 7.99, 7.99, 1.99...","""If you liked City Island and other early simu...",Sparkling Society,4+,EN,139537408.0,Games,"Games, Strategy, Simulation, Entertainment",27/03/2014,17/10/2018,Intermediate


# predefined function 

## number of words

In [212]:
def calculate_name_words_number(data):
  y=data['Name'].str.replace('[^\w\s]', '').str.split(':| ').apply(lambda x: [str(s) for s in x])
  listy=[]
  for item in y:
    listy.append(len(item)+1)
  return listy

In [213]:
def calculate_subtitle_words_number(data):
  mydata=data['Subtitle'].fillna('one')
  y=mydata.str.replace('[^\w\s]', '', regex=True).str.split(':| ').apply(lambda x: [str(s) for s in x])
  listy=[]
  one=['one']
  for item in y:
    if item == one:
      listy.append(len(item))
    else:
            listy.append(len(item)+1)
  return listy

## price

In [214]:
def has_price_feature(data):
  lsty=[]
  for item in data['Price']:
    if item == 0 :
      lsty.append(0)
    else:
      lsty.append(1)
  return lsty

## encode

In [215]:
def encode_rate(data):
  return data['Rate'].replace('Low',0).replace('Intermediate',1).replace('High',2)


In [216]:
def encode_subtitle(data):
  listy=[]
  mydata=data['Subtitle'].fillna(-11)
  for row in mydata:
    if row == -11 :
      listy.append(0)
    else:
      listy.append(1)

  return listy

In [217]:
def encode_age_rating(data):

    data['Age Rating']=data['Age Rating'].apply(lambda x : x.replace("+", ""))
    data['Age Rating']=data['Age Rating'].apply(lambda x : int(x))
    return data['Age Rating']
    

## outlier

In [218]:
def check_outliar(data):
  # Calculate the interquartile range (IQR) of the data
  Q1 = data.quantile(0.25)
  Q3 =data.quantile(0.75)
  IQR = Q3 - Q1

  # Determine the lower and upper bounds for outliers
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  outliers = data[(data < lower_bound) | (data > upper_bound)]

  # Print the outliers
  return (len(outliers))

In [219]:
def handle_outlier(data):
  # Create a sample dataset
  thedata = data

  # Set the Winsorization percentiles
  pct = 0.20

  # Calculate the percentile values
  low_val = np.percentile(thedata, pct * 100)
  high_val = np.percentile(thedata, (1 - pct) * 100)

  # Replace the extreme values with the percentile values
  thedata[data < low_val] = low_val
  thedata[data > high_val] = high_val

  # Print the transformed data
  # print(data)
  data=thedata

## visual_appeal

In [220]:
def calc_appeal_score(url):
    # time.sleep(1)
    if pd.isnull(url) :
      return None
      
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")

    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    # Resize image
    target_size = (256, 256)  # specify target size

    resized_image = cv2.resize(image, target_size)

    # Convert image to grayscale
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Compute color histogram
    hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])

    # Normalize histogram
    hist /= hist.sum()

    # Calculate visual appeal score based on histogram
    visual_appeal_score = sum([hist[i] * (i / 256) for i in range(256)])
    return visual_appeal_score
    # Print visual appeal score
    # print("Visual Appeal Score:", visual_appeal_score)

In [221]:
def icon_url_feature_eng(data):
  mylist=data['Icon URL']
  mylist=mylist.apply(calc_appeal_score)
  data['visual_appeal']=mylist

In [222]:
def convert_visual_appeal(data):
  listo=data['visual_appeal']
  listo.fillna('[0.0]',inplace=True)
  listo=listo.apply(lambda x : str(x))
  listo=listo.apply(lambda x : x.replace("[", "").replace("]", ""))
  listo=listo.apply(lambda x : float(x))
  mymean=0.4856552140087041 # mean visual_appeal for xtrain
  listo = [mymean if x == 0 else x for x in listo]
  return listo

## url 

In [223]:
def check_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return True
        else:
            return False
    except:
        return False

In [224]:
def check_url_feature(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return 1
        else:
            return 0
    except:
        return 0


## in_app_purchases

In [225]:
def purchases_or_not_feature(data):
  lsty=[]
  for item in data['In-app Purchases']:
    if item =='0.0':
      lsty.append(0)
    else:
      lsty.append(1)
  return lsty


In [226]:
def in_app_purchase_eng(data):
  data['In-app Purchases']=data['In-app Purchases'].str.split(',').apply(lambda x: [float(s) for s in x])
  meanc=[]
  medianc=[]
  maxc=[]
  minc=[]
  for row in data['In-app Purchases']:
    meanc.append(np.mean(row))
    medianc.append(np.median(row))
    maxc.append(np.max(row))
    minc.append(np.min(row))
  return meanc,medianc,maxc,minc

## drop

In [227]:
def drop_col(data):
  droped=['Description','Developer','ID','In-app Purchases','Subtitle','URL','Languages','Icon URL','Primary Genre','Genres','Original Release Date','Current Version Release Date']
  data.drop(columns=droped,inplace=True)

## dates

In [228]:
def convert_date_to_numeric_timestamp(dates):
    timestamps = []
    for date in dates:
        dt_obj = datetime.strptime(date, '%d/%m/%Y')
        timestamp = datetime.timestamp(dt_obj)
        timestamps.append(timestamp)
    return timestamps

In [229]:
def convert_date(data):
  data=data.apply(pd.Timestamp)
  day=data.dt.day
  month=data.dt.month
  year=data.dt.year

  return day,month,year

In [230]:
def calculate_app_ages(data):
  cday,cmonth,cyear=convert_date(data['Current Version Release Date'])
  oday,omonth,oyear=convert_date(data['Original Release Date'])
  cday=list(cday)
  cmonth=list(cmonth)
  cyear=list(cyear)
  oday=list(oday)
  omonth=list(omonth)
  oyear=list(oyear)

  days=[]
  months=[]
  years=[]
  testc,testo=0,0
  for i in range(len(cyear)):
    tmp_day=((datetime(cyear[i], cmonth[i], cday[i]))-(datetime(oyear[i], omonth[i], oday[i]))).days
    if tmp_day < 0:
      tmp_day= tmp_day * (- 1.0) 
    tmp_month=tmp_day/30.5
    tmp_month = "{:.{prec}f}".format(tmp_month, prec=3)
    tmp_month=float(tmp_month)
    if tmp_month < 0:
      tmp_month= tmp_month * (- 1.0)

    tmp_year=tmp_day/365.25
    tmp_year="{:.{prec}f}".format(tmp_year, prec=3)
    tmp_year=float(tmp_year)
    if tmp_year < 0:
      tmp_year= tmp_year * (- 1.0)

    days.append(tmp_day)
    months.append(tmp_month)
    years.append(tmp_year)

  return years, months,days

## App count and density

In [231]:
def dev_app_count_col(data):

  filename = "developer_apps_count.joblib"
  developer_apps_count=joblib.load(filename)

  listo=[]
  for dev in data['Developer']:

    try :
      listo.append(developer_apps_count.loc[dev,'App Count'])
    except:
      listo.append(1)

  return listo



In [232]:
def calc_density(data):
  data['density']=data['User Rating Count']/data['Size']

## description -> canceled

In [233]:
def description_features(data):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    punc = '''!()-[]{};:'"\,<>/?@#$%^&*_~'''


    Preprocessed_Description_before_stopwords=[]
    Preprocessed_Description_after_stopwords=[]

    num_of_words_before_remove_stopwords =[]
    num_of_words_after_remove_stopwords =[]


    percentage_list=[]
    lemmatizer=WordNetLemmatizer()
    for index,row in data.iterrows():
        sentence=row['Description']
    #cleaning the Descriptions from newline char and the punctuation and links and the numbers and garbadge data
        sentence = sentence.replace("\\n","\n")
        sentence = sentence.replace("\n", " ")
        sentence = re.sub(r'u[\da-f]{4}', '', sentence)
        sentence = re.sub(r'http\S+', '', sentence)
        sentence = re.sub(r'\d+', '', sentence)
        sentence_without_punc = ''.join([ele for ele in sentence if ele not in punc])
    #Apply the Word Tokanization
        word_tokens=word_tokenize(sentence_without_punc)
    #Removing the . char from the Tokens
        for item in word_tokens:
            if '.' in item:
                word_tokens.remove(item)

    #Count the number of words before removing the stopwords
        num_of_words_before_remove_stopwords.append(len(word_tokens)) 


    #Append All Preprocessed tokens before removing stop words in a new list called Preprocessed_Description
        Preprocessed_Description_before_stopwords.append(word_tokens)


    #Removing the StopWords from tokens
        tokens_without_stopwords = [word for word in word_tokens if word.casefold() not in stop_words]

    #Count the number of words after removing the stopwords
        num_of_words_after_remove_stopwords.append(len(tokens_without_stopwords))

    #Applying the Lemmatizations on the tokens
        lemmatized_words = [lemmatizer.lemmatize(word,pos='v') for word in tokens_without_stopwords]

    #Append All Preprocessed tokens after removing stop words in a new list called Preprocessed_Description
        Preprocessed_Description_after_stopwords.append(lemmatized_words)
    #Putting the defualt value (1) as this description does not have any words and measure the precentage between the number of words after removing the stopwords and between the words befor removing the stopwords
    for i in range(0,len(num_of_words_after_remove_stopwords)):
        if (num_of_words_after_remove_stopwords[i]==0 or num_of_words_before_remove_stopwords[i]==0):
            num_of_words_after_remove_stopwords[i]= num_of_words_after_remove_stopwords[i]+1
            num_of_words_before_remove_stopwords[i]= num_of_words_before_remove_stopwords[i]+1
        percentage_list.append((num_of_words_after_remove_stopwords[i]/num_of_words_before_remove_stopwords[i]))
      #data.loc[index,'Description']=preprocessing_Des 
    data['length_of_all_discription']=num_of_words_before_remove_stopwords
    data['length_of_discription_summary']=num_of_words_after_remove_stopwords
    data['discription_summary_percent']=percentage_list


## fill nulls

In [234]:
def fill_nulls(data):
  org_rd='2/9/2016'
  curr_rd='31/07/2019'

  if data['Name'].isna().sum()>0: #mode of xtrain word number
     data['Name'].fillna('a c g',inplace=True)
     
  # subtitle -> filled in another function

  if data['User Rating Count'].isna().sum()>0: #mode of xtrain 
     data['User Rating Count'].fillna(5,inplace=True)

  if data['Price'].isna().sum()>0:  #mode of xtrain 
     data['Price'].fillna(0.00,inplace=True)

  if data['In-app Purchases'].isna().sum()>0: # this means no purchases 
     data['In-app Purchases'].fillna('0.0',inplace=True)

  if data['Description'].isna().sum()>0:
     data['Description'].fillna('happy',inplace=True)
  
  if data['Developer'].isna().sum()>0: #mode of xtrain 
     data['Developer'].fillna('Tapps Tecnologia da Informa\xe7\xe3o Ltda.',inplace=True)
  
  if data['Age Rating'].isna().sum()>0: #mode of xtrain
     data['Age Rating'].fillna('4+',inplace=True)

  if data['Languages'].isna().sum()>0:
     data['Languages'].fillna('EN',inplace=True)

  if data['Size'].isna().sum()>0: #median of xtrain
     data['Size'].fillna(67060736.0,inplace=True)
    
  if data['Primary Genre'].isna().sum()>0: #mode of xtrain
     data['Primary Genre'].fillna('Games',inplace=True)

  if data['Genres'].isna().sum()>0: # mode of xtrain
     data['Genres'].fillna('Games, Strategy, Puzzle, Simulation, Action',inplace=True)


  if data['Original Release Date'].isna().sum()>0 : #mode of xtrain 
     data['Original Release Date'].fillna(org_rd,inplace=True)

  if data['Current Version Release Date'].isna().sum()>0 : #mode of xtrain 
     data['Current Version Release Date'].fillna(curr_rd,inplace=True)


## Description

In [235]:
def Cleaning_Descriptions(sentence):
    punc = '''!()-[]{};:'"\,<>/?@#$%^&*_~'''
    sentence = sentence.replace("\\n","\n")
    sentence = sentence.replace("\n"," ")
    sentence = re.sub(r'u[\da-f]{4}','', sentence)
    sentence = re.sub(r'http\S+','', sentence)
    sentence = re.sub(r'\d+','', sentence)
    return ''.join([ele for ele in sentence if ele not in punc])


In [236]:
def remove_stopwords(text):
    word_tokens=word_tokenize(text)
    return ' '.join([word for word in word_tokens if word not in stop_words])

In [237]:
def remove_additional_space(text):
    text=re.sub('\s+',' ',text)
    return text

In [238]:
def Summarization(text):
    stopWords = set(stopwords.words("english")) 
    words = word_tokenize(text) 
    freqTable = dict() 
    for word in words: 
        if word in stopWords: 
            continue
        if word in freqTable: 
            freqTable[word] += 1
        else: 
            freqTable[word] = 1
    sentences = sent_tokenize(text) 
    sentenceValue = dict() 

    for sentence in sentences: 
        for word, freq in freqTable.items(): 
            if word in sentence.lower(): 
                if sentence in sentenceValue: 
                    sentenceValue[sentence] += freq 
                else: 
                    sentenceValue[sentence] = freq 
    sumValues = 0
    for sentence in sentenceValue: 
        sumValues += sentenceValue[sentence] 
    average = int(sumValues / (len(sentenceValue)+1))
    summary = '' 
    for sentence in sentences: 
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)): 
            summary += " " + sentence 
    return summary

In [239]:
stemmer=PorterStemmer()
def stemm_words(text):
    word_tokens=word_tokenize(text)
    return " ".join([stemmer.stem(word) for word in word_tokens])

In [240]:
lemmatizer = WordNetLemmatizer()
wordnet_map={"N":wordnet.NOUN,"V":wordnet.VERB,"J":wordnet.ADJ,"R":wordnet.ADV}
def lemmatize_words(text):
    word_tokens=word_tokenize(text)
    pos_text=pos_tag(word_tokens)
    return " ".join([lemmatizer.lemmatize(word,wordnet_map.get(pos[0],wordnet.NOUN)) for word,pos in pos_text])

# fill nulls 

In [241]:
fill_nulls(data)

# encode rate

In [242]:
data['Rate']=encode_rate(data)
temp=data['Rate']
data.drop(columns= 'Rate',inplace=True)
data['Rate']=temp
data

,URL,ID,Name,Subtitle,Icon URL,User Rating Count,Price,In-app Purchases,Description,Developer,Age Rating,Languages,Size,Primary Genre,Genres,Original Release Date,Current Version Release Date,Rate
0,https://apps.apple.com/us/app/my-colony/id1123...,1123806108,My Colony,NaN,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,682.0,0.00,4.99,My Colony is a brand new space colonization si...,Brandon Stecklein,4+,EN,131994624.0,Games,"Games, Simulation, Strategy",18/10/2016,2/7/2019,1
1,https://apps.apple.com/us/app/hidden-object-co...,893444350,Hidden Object Countryside Love,Casual puzzles full of fun!,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,19.0,0.00,"0.99, 1.99, 4.99, 9.99",Ready to start a journey full of fun and surpr...,Zheng Hui,4+,"EN, FR, DE, JA, RU, ZH, ES, ZH",47676416.0,Games,"Games, Strategy, Puzzle, Entertainment",17/10/2014,15/12/2018,1
2,https://apps.apple.com/us/app/haunted-towns-hi...,1122711408,Haunted Towns Hidden Object \u2013 Secret Myst...,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,9.0,0.99,0.0,Hidden Objects \u2013 Haunted Towns is an arti...,Detention Apps,4+,EN,65063936.0,Games,"Games, Strategy, Puzzle, Entertainment",10/6/2016,10/6/2016,1
3,https://apps.apple.com/us/app/battlepillars-mu...,664303561,Battlepillars: Multiplayer (PVP) Real Time Str...,NaN,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,1155.0,0.00,"0.99, 0.99, 1.99, 1.99, 4.99, 4.99, 9.99, 2.99...",Crawl into the bug battle with this action pac...,"Hitcents.com, Inc.",9+,EN,113438720.0,Games,"Games, Entertainment, Strategy, Action",2/10/2013,24/09/2015,2
4,https://apps.apple.com/us/app/chess-genius/id2...,294631172,Chess Genius,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,156.0,5.99,0.0,"""ChessGenius, award winning chess program. Sup...",Lang Software Limited,4+,"EN, FR, DE, IT, ES",1751040.0,Games,"Games, Strategy, Board, Entertainment",8/11/2008,5/8/2016,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,https://apps.apple.com/us/app/antiquitas/id131...,1310689005,Antiquitas,Build your own ancient city!,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,26.0,0.00,2.99,"""Antiquitas is an ancient city builder and sim...",Brandon Stecklein,12+,EN,145886208.0,Games,"Games, Strategy, Simulation",17/11/2017,8/7/2019,1
650,https://apps.apple.com/us/app/simple-chess-boa...,531158560,Simple chess board,NaN,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,26.0,0.00,0.99,"""A simple chess board and a chess clock withou...",Luis Lopez,4+,"CA, EN, ES",7204864.0,Games,"Games, Strategy, Board, Utilities",5/7/2012,10/11/2015,2
651,https://apps.apple.com/us/app/wheres-my-jewel/...,689905568,"""Where's My Jewel?""",NaN,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,10.0,0.00,2.99,Download this fun jewel matching puzzle game.\...,Makeover Mania Story Games,4+,EN,12992512.0,Games,"Games, Strategy, Puzzle",24/08/2013,24/08/2013,1
652,https://apps.apple.com/us/app/city-island-2-bu...,838193156,City Island 2: Building Story,Fun Offline City Building Game,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,652.0,0.00,"2.99, 2.99, 7.99, 2.99, 1.99, 7.99, 7.99, 1.99...","""If you liked City Island and other early simu...",Sparkling Society,4+,EN,139537408.0,Games,"Games, Strategy, Simulation, Entertainment",27/03/2014,17/10/2018,1


# **feature engineering befor splitting**



## has purchases 

In [243]:
data['has_purchases']=purchases_or_not_feature(data)



## game link work

In [244]:
# with open('/content/game_working_urls.pkl', 'rb') as f:
#     # Load the pickled data into a Python object
#     my_list = pickle.load(f)
# data['link_work']=my_list

## encode Age Rating

In [245]:
data['Age Rating']=encode_age_rating(data)
data['Age Rating']

0       4
1       4
2       4
3       9
4       4
       ..
649    12
650     4
651     4
652     4
653     9
Name: Age Rating, Length: 653, dtype: int64

## apply feature engineering on in-app-purchases 

In [246]:
data['mean Purchases'],data['meedian Purchases'],data['max Purchases'],data['min Purchases']=in_app_purchase_eng(data)


## calculate number of words per game name

In [247]:
# calculate number of words per game name
data['Name']=calculate_name_words_number(data)


<ipython-input-212-6149ac8e48d2>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  y=data['Name'].str.replace('[^\w\s]', '').str.split(':| ').apply(lambda x: [str(s) for s in x])


## calculate density 

In [248]:
# calculate density -> User Rating Count/ game Size
calc_density(data)

## convert date to numeric time stamp

In [249]:
data['Curr_datestamp']=convert_date_to_numeric_timestamp(data['Current Version Release Date'])
data['orig_datestamp']=convert_date_to_numeric_timestamp(data['Original Release Date'])
data

,URL,ID,Name,Subtitle,Icon URL,User Rating Count,Price,In-app Purchases,Description,Developer,...,Current Version Release Date,Rate,has_purchases,mean Purchases,meedian Purchases,max Purchases,min Purchases,density,Curr_datestamp,orig_datestamp
0,https://apps.apple.com/us/app/my-colony/id1123...,1123806108,3,NaN,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,682.0,0.00,[4.99],My Colony is a brand new space colonization si...,Brandon Stecklein,...,2/7/2019,1,1,4.99,4.99,4.99,4.99,5.166877e-06,1.562026e+09,1.476749e+09
1,https://apps.apple.com/us/app/hidden-object-co...,893444350,5,Casual puzzles full of fun!,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,19.0,0.00,"[0.99, 1.99, 4.99, 9.99]",Ready to start a journey full of fun and surpr...,Zheng Hui,...,15/12/2018,1,1,4.49,3.49,9.99,0.99,3.985199e-07,1.544832e+09,1.413504e+09
2,https://apps.apple.com/us/app/haunted-towns-hi...,1122711408,16,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,9.0,0.99,[0.0],Hidden Objects \u2013 Haunted Towns is an arti...,Detention Apps,...,10/6/2016,1,0,0.00,0.00,0.00,0.00,1.383255e-07,1.465517e+09,1.465517e+09
3,https://apps.apple.com/us/app/battlepillars-mu...,664303561,7,NaN,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,1155.0,0.00,"[0.99, 0.99, 1.99, 1.99, 4.99, 4.99, 9.99, 2.9...",Crawl into the bug battle with this action pac...,"Hitcents.com, Inc.",...,24/09/2015,2,1,3.99,2.49,9.99,0.99,1.018171e-05,1.443053e+09,1.380672e+09
4,https://apps.apple.com/us/app/chess-genius/id2...,294631172,3,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,156.0,5.99,[0.0],"""ChessGenius, award winning chess program. Sup...",Lang Software Limited,...,5/8/2016,1,0,0.00,0.00,0.00,0.00,8.908991e-05,1.470355e+09,1.226102e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,https://apps.apple.com/us/app/antiquitas/id131...,1310689005,2,Build your own ancient city!,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,26.0,0.00,[2.99],"""Antiquitas is an ancient city builder and sim...",Brandon Stecklein,...,8/7/2019,1,1,2.99,2.99,2.99,2.99,1.782211e-07,1.562544e+09,1.510877e+09
650,https://apps.apple.com/us/app/simple-chess-boa...,531158560,4,NaN,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,26.0,0.00,[0.99],"""A simple chess board and a chess clock withou...",Luis Lopez,...,10/11/2015,2,1,0.99,0.99,0.99,0.99,3.608673e-06,1.447114e+09,1.341446e+09
651,https://apps.apple.com/us/app/wheres-my-jewel/...,689905568,4,NaN,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,10.0,0.00,[2.99],Download this fun jewel matching puzzle game.\...,Makeover Mania Story Games,...,24/08/2013,1,1,2.99,2.99,2.99,2.99,7.696741e-07,1.377302e+09,1.377302e+09
652,https://apps.apple.com/us/app/city-island-2-bu...,838193156,6,Fun Offline City Building Game,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,652.0,0.00,"[2.99, 2.99, 7.99, 2.99, 1.99, 7.99, 7.99, 1.9...","""If you liked City Island and other early simu...",Sparkling Society,...,17/10/2018,1,1,4.79,2.99,7.99,1.99,4.672582e-06,1.539734e+09,1.395878e+09


## encode dates to year,month,day

In [250]:
data["orgDay"],data["orgMonth"],data["orgYear"]=convert_date(data["Original Release Date"])
data["currDay"],data["currMonth"],data["currYear"]=convert_date(data["Current Version Release Date"])


<ipython-input-229-1b575e1ae60e>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data=data.apply(pd.Timestamp)
<ipython-input-229-1b575e1ae60e>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data=data.apply(pd.Timestamp)


## calculate age of the app from the date 

In [251]:
data['app age in years'],data['app age in months'],data['app age in days']=calculate_app_ages(data)
data[['app age in years','app age in months','app age in days']]


<ipython-input-229-1b575e1ae60e>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data=data.apply(pd.Timestamp)


,app age in years,app age in months,app age in days
0,2.305,27.607,842.0
1,4.162,49.836,1520.0
2,0.000,0.000,0.0
3,2.617,31.344,956.0
4,7.740,92.689,2827.0
...,...,...,...
649,1.719,20.590,628.0
650,3.428,41.049,1252.0
651,0.000,0.000,0.0
652,4.559,54.590,1665.0


## has price feature

In [252]:
data['has_price']=has_price_feature(data)


## has subtitle feature 

In [253]:
data['has_subtitle']=encode_subtitle(data)

data['has_subtitle']

0      0
1      1
2      0
3      0
4      0
      ..
649    1
650    0
651    0
652    1
653    1
Name: has_subtitle, Length: 653, dtype: int64

## calculate subtitle words number 

In [254]:
data['subtitle_words_number']=calculate_subtitle_words_number(data)


# split to x and y

In [255]:
y=data['Rate']
data=data.drop(columns='Rate')
y.shape

(653,)

# feature enginerring after splitting

## calculate number of apps for every developer 

In [256]:
# number of app for developers to xtrain and xtest
data['dev_apps_count']=dev_app_count_col(data)


# handel categorical features 

In [257]:

data["Languages"]=data["Languages"].str.split(', ')
data["Genres"]=data["Genres"].str.split(', ')


In [258]:
lang_mlb=joblib.load("/content/lang_mlb.joblib")
Genres_mlb=joblib.load("/content/Genres_mlb.joblib")
cat_encoder=joblib.load("/content/cat_encoder.joblib")

encode language


In [259]:
expandedLabelData_lang = lang_mlb.transform(data.Languages)

languages_df = pd.DataFrame(expandedLabelData_lang, columns=lang_mlb.classes_)
languages_df

,AF,AM,AR,AS,AY,AZ,BE,BG,BN,BO,...,TO,TR,TT,UG,UK,UR,UZ,VI,YI,ZH
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
649,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
650,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
651,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


encode genres 


In [260]:
expandedLabelData_genres = Genres_mlb.transform(data.Genres)
genres_df = pd.DataFrame(expandedLabelData_genres, columns=Genres_mlb.classes_)
genres_df

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Emoji & Expressions'] will be ignored
  warnings.warn(


,Action,Adventure,Board,Books,Business,Card,Casino,Casual,Education,Entertainment,...,Shopping,Simulation,Social Networking,Sports,Stickers,Strategy,Travel,Trivia,Utilities,Word
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
649,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
650,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
651,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,1,0,0,0,0


encode primary genre

In [261]:

# fit and transform the "Primary Genre" column
ohe_Primary_Genre = cat_encoder.transform(data[['Primary Genre']])

# create a new DataFrame with the one-hot encoded columns
encoded_PrimaryGenre = pd.DataFrame(ohe_Primary_Genre.toarray(), columns=cat_encoder.categories_)

# df_encoded = df_encoded.reset_index(level=1)
encoded_PrimaryGenre = encoded_PrimaryGenre.reset_index(level=0, drop=True)

# map the column index to a string
encoded_PrimaryGenre.columns = encoded_PrimaryGenre.columns.map(lambda x: x[0])


encoded_PrimaryGenre


,Book,Business,Education,Entertainment,Finance,Food & Drink,Games,Health & Fitness,Lifestyle,Medical,Music,News,Productivity,Reference,Shopping,Social Networking,Sports,Stickers,Utilities
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
649,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
650,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
651,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


concatenating genres & language with xtrain and xtest

In [262]:
data=data.reset_index(level=0, drop=True)
data=pd.concat([data, genres_df,languages_df], axis=1)
data.shape

(653, 188)

adding unexisted primary genre as features and summing existed one in xtrain and xtest

In [263]:
for item in (encoded_PrimaryGenre.columns):
  if item in genres_df:
    listy=[]
    for i in range(len(data)):
      listy.append(data[item][i]+ encoded_PrimaryGenre[item][i])    
   
    data[item]=listy 
  else:
    data[item]=encoded_PrimaryGenre[item].values
data.shape

(653, 189)

# target encoding 

In [264]:
target_column=joblib.load( "/content/target_column.joblib")
terget_encoder=pd.read_pickle("/content/target_encoder.pkl")

In [265]:

target_train=terget_encoder.transform(data[target_column].applymap(lambda x: tuple(x) if isinstance(x, list) else x))
data["Primary_Genre"]=target_train['Primary Genre'].values
data["Developer_target"]=target_train['Developer'].values
data["Age_Rating_target"]=target_train['Age Rating'].values
data


,URL,ID,Name,Subtitle,Icon URL,User Rating Count,Price,In-app Purchases,Description,Developer,...,UK,UR,UZ,VI,YI,ZH,Book,Primary_Genre,Developer_target,Age_Rating_target
0,https://apps.apple.com/us/app/my-colony/id1123...,1123806108,3,NaN,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,682.0,0.00,[4.99],My Colony is a brand new space colonization si...,Brandon Stecklein,...,0,0,0,0,0,0,0.0,1.418599,1.352034,4
1,https://apps.apple.com/us/app/hidden-object-co...,893444350,5,Casual puzzles full of fun!,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,19.0,0.00,"[0.99, 1.99, 4.99, 9.99]",Ready to start a journey full of fun and surpr...,Zheng Hui,...,0,0,0,0,0,1,0.0,1.418599,1.492283,4
2,https://apps.apple.com/us/app/haunted-towns-hi...,1122711408,16,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,9.0,0.99,[0.0],Hidden Objects \u2013 Haunted Towns is an arti...,Detention Apps,...,0,0,0,0,0,0,0.0,1.418599,1.782263,4
3,https://apps.apple.com/us/app/battlepillars-mu...,664303561,7,NaN,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,1155.0,0.00,"[0.99, 0.99, 1.99, 1.99, 4.99, 4.99, 9.99, 2.9...",Crawl into the bug battle with this action pac...,"Hitcents.com, Inc.",...,0,0,0,0,0,0,0.0,1.418599,1.357285,9
4,https://apps.apple.com/us/app/chess-genius/id2...,294631172,3,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,156.0,5.99,[0.0],"""ChessGenius, award winning chess program. Sup...",Lang Software Limited,...,0,0,0,0,0,0,0.0,1.418599,1.416344,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,https://apps.apple.com/us/app/antiquitas/id131...,1310689005,2,Build your own ancient city!,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,26.0,0.00,[2.99],"""Antiquitas is an ancient city builder and sim...",Brandon Stecklein,...,0,0,0,0,0,0,0.0,1.418599,1.352034,12
649,https://apps.apple.com/us/app/simple-chess-boa...,531158560,4,NaN,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,26.0,0.00,[0.99],"""A simple chess board and a chess clock withou...",Luis Lopez,...,0,0,0,0,0,0,0.0,1.418599,1.499136,4
650,https://apps.apple.com/us/app/wheres-my-jewel/...,689905568,4,NaN,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,10.0,0.00,[2.99],Download this fun jewel matching puzzle game.\...,Makeover Mania Story Games,...,0,0,0,0,0,0,0.0,1.418599,1.286360,4
651,https://apps.apple.com/us/app/city-island-2-bu...,838193156,6,Fun Offline City Building Game,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,652.0,0.00,"[2.99, 2.99, 7.99, 2.99, 1.99, 7.99, 7.99, 1.9...","""If you liked City Island and other early simu...",Sparkling Society,...,0,0,0,0,0,0,0.0,1.418599,1.449558,4


# description column feature extraction

## Cleaning 

In [266]:
data['Description']=data['Description'].apply(lambda x:Cleaning_Descriptions(x))
data['Description']=data['Description'].str.lower()
data

,URL,ID,Name,Subtitle,Icon URL,User Rating Count,Price,In-app Purchases,Description,Developer,...,UK,UR,UZ,VI,YI,ZH,Book,Primary_Genre,Developer_target,Age_Rating_target
0,https://apps.apple.com/us/app/my-colony/id1123...,1123806108,3,NaN,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,682.0,0.00,[4.99],my colony is a brand new space colonization si...,Brandon Stecklein,...,0,0,0,0,0,0,0.0,1.418599,1.352034,4
1,https://apps.apple.com/us/app/hidden-object-co...,893444350,5,Casual puzzles full of fun!,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,19.0,0.00,"[0.99, 1.99, 4.99, 9.99]",ready to start a journey full of fun and surpr...,Zheng Hui,...,0,0,0,0,0,1,0.0,1.418599,1.492283,4
2,https://apps.apple.com/us/app/haunted-towns-hi...,1122711408,16,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,9.0,0.99,[0.0],hidden objects haunted towns is an artistical...,Detention Apps,...,0,0,0,0,0,0,0.0,1.418599,1.782263,4
3,https://apps.apple.com/us/app/battlepillars-mu...,664303561,7,NaN,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,1155.0,0.00,"[0.99, 0.99, 1.99, 1.99, 4.99, 4.99, 9.99, 2.9...",crawl into the bug battle with this action pac...,"Hitcents.com, Inc.",...,0,0,0,0,0,0,0.0,1.418599,1.357285,9
4,https://apps.apple.com/us/app/chess-genius/id2...,294631172,3,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,156.0,5.99,[0.0],chessgenius award winning chess program. super...,Lang Software Limited,...,0,0,0,0,0,0,0.0,1.418599,1.416344,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,https://apps.apple.com/us/app/antiquitas/id131...,1310689005,2,Build your own ancient city!,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,26.0,0.00,[2.99],antiquitas is an ancient city builder and simu...,Brandon Stecklein,...,0,0,0,0,0,0,0.0,1.418599,1.352034,12
649,https://apps.apple.com/us/app/simple-chess-boa...,531158560,4,NaN,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,26.0,0.00,[0.99],a simple chess board and a chess clock without...,Luis Lopez,...,0,0,0,0,0,0,0.0,1.418599,1.499136,4
650,https://apps.apple.com/us/app/wheres-my-jewel/...,689905568,4,NaN,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,10.0,0.00,[2.99],download this fun jewel matching puzzle game. ...,Makeover Mania Story Games,...,0,0,0,0,0,0,0.0,1.418599,1.286360,4
651,https://apps.apple.com/us/app/city-island-2-bu...,838193156,6,Fun Offline City Building Game,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,652.0,0.00,"[2.99, 2.99, 7.99, 2.99, 1.99, 7.99, 7.99, 1.9...",if you liked city island and other early simul...,Sparkling Society,...,0,0,0,0,0,0,0.0,1.418599,1.449558,4


## Removing the stop words

In [267]:
data['Description']=data['Description'].apply(lambda x:remove_stopwords(x))
data['Description'].head()

0    colony brand new space colonization simulation...
1    ready start journey full fun surprise download...
2    hidden objects haunted towns artistically craf...
3    crawl bug battle action packed real time strat...
4    chessgenius award winning chess program . supe...
Name: Description, dtype: object

## Remove Any additional Whitespace

In [268]:
data['Description']=data['Description'].apply(lambda x:remove_additional_space(x))
data.head(5)

,URL,ID,Name,Subtitle,Icon URL,User Rating Count,Price,In-app Purchases,Description,Developer,...,UK,UR,UZ,VI,YI,ZH,Book,Primary_Genre,Developer_target,Age_Rating_target
0,https://apps.apple.com/us/app/my-colony/id1123...,1123806108,3,NaN,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,682.0,0.00,[4.99],colony brand new space colonization simulation...,Brandon Stecklein,...,0,0,0,0,0,0,0.0,1.418599,1.352034,4
1,https://apps.apple.com/us/app/hidden-object-co...,893444350,5,Casual puzzles full of fun!,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,19.0,0.00,"[0.99, 1.99, 4.99, 9.99]",ready start journey full fun surprise download...,Zheng Hui,...,0,0,0,0,0,1,0.0,1.418599,1.492283,4
2,https://apps.apple.com/us/app/haunted-towns-hi...,1122711408,16,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,9.0,0.99,[0.0],hidden objects haunted towns artistically craf...,Detention Apps,...,0,0,0,0,0,0,0.0,1.418599,1.782263,4
3,https://apps.apple.com/us/app/battlepillars-mu...,664303561,7,NaN,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,1155.0,0.00,"[0.99, 0.99, 1.99, 1.99, 4.99, 4.99, 9.99, 2.9...",crawl bug battle action packed real time strat...,"Hitcents.com, Inc.",...,0,0,0,0,0,0,0.0,1.418599,1.357285,9
4,https://apps.apple.com/us/app/chess-genius/id2...,294631172,3,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,156.0,5.99,[0.0],chessgenius award winning chess program . supe...,Lang Software Limited,...,0,0,0,0,0,0,0.0,1.418599,1.416344,4


## Summarization 

In [269]:
data['Description'][0]

'colony brand new space colonization simulation strategy game maker hit simulation games planet land business empire colony never one story ville type app purchase nickel dime games . aim futuristic take classic sim city type game . plan adding multiplayer enemies things like . really want keep game casual scifi simulation category . currently colony early days active development open feedback suggestions users like . please try game leave comments feedback help shape epic new simulation unfold'

In [270]:
data['Description']=data['Description'].apply(lambda x:Summarization(x))
data['Description'][0]

' colony brand new space colonization simulation strategy game maker hit simulation games planet land business empire colony never one story ville type app purchase nickel dime games .'

## Applying the Stemming

In [271]:
data['Description']=data['Description'].apply(lambda x:stemm_words(x))
data['Description'].head()

0    coloni brand new space colon simul strategi ga...
1    readi start journey full fun surpris download ...
2    fun object quest challeng hidden object puzzl ...
3    crawl bug battl action pack real time strategi...
4    pack featur beginn expert simpl eleg user inte...
Name: Description, dtype: object

## Applying Lemmatization and POS tagging

In [272]:
data['Description']=data['Description'].apply(lambda x:lemmatize_words(x))
data['Description'].head()

0    coloni brand new space colon simul strategi ga...
1    readi start journey full fun surpris download ...
2    fun object quest challeng hidden object puzzl ...
3    crawl bug battl action pack real time strategi...
4    pack featur beginn expert simpl eleg user inte...
Name: Description, dtype: object

## Feature Extraction using Bag of words

In [273]:
cv=joblib.load( "/content/count_vectorizer.joblib")


In [274]:
features=cv.transform(data['Description'])
data_f_train=pd.DataFrame(features.toarray(),columns=cv.get_feature_names_out())
data_f_train

,aa,aaa,aaaaaaa,aadec,aan,aaron,aat,ab,abad,abadi,...,zumi,zusehend,zuzi,zviru,zvsa,zwischen,zxerov,zxfcrich,zynga,zzed
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
649,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
650,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
651,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## concatenaing  dataframes

In [275]:
# total_des=joblib.load( "/content/target_column.joblib")

# data_f_train=data_f_train[total_des]
# data_f_train

In [276]:
# df_encoded = df_encoded.reset_index(level=1)
data_f_train = data_f_train.reset_index(level=0, drop=True)

data=data.reset_index(level=0, drop=True)
data=pd.concat([data,data_f_train], axis=1)
data.shape

(653, 17360)

In [277]:
selected_columns=joblib.load( "/content/selected_columns.joblib")

data=data[selected_columns]
data.shape

(653, 225)

# standardization for xtrain and xtest

In [278]:
standard_scaler=joblib.load("/content/standard_scaler.joblib")


In [279]:
scaled_data=standard_scaler.transform(data)
scaled_data

array([[-0.20060271, -0.01555114, -0.39490891, ..., -0.11309366,
        -0.0249785 , -0.01555114],
       [-0.20060271, -0.01555114, -0.39490891, ..., -0.11309366,
        -0.0249785 , -0.01555114],
       [-0.20060271, -0.01555114, -0.39490891, ..., 10.70205978,
        -0.0249785 , -0.01555114],
       ...,
       [-0.20060271, -0.01555114, -0.39490891, ..., -0.11309366,
        -0.0249785 , -0.01555114],
       [-0.20060271, -0.01555114, -0.39490891, ..., -0.11309366,
        -0.0249785 , -0.01555114],
       [-0.20060271, -0.01555114, -0.39490891, ..., -0.11309366,
        -0.0249785 , -0.01555114]])

# **models**

## logestic regrrestion

In [280]:
lr=joblib.load( "/content/logestic.joblib")


In [281]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y_pred = lr.predict(scaled_data)
ytest=y

print("Test Accuracy:", accuracy_score(ytest, y_pred))
cm = confusion_matrix(ytest, y_pred)
precision = precision_score(ytest, y_pred, average='weighted')
recall = recall_score(ytest, y_pred, average='weighted')
f1 = f1_score(ytest, y_pred, average='weighted')
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion matrix:")
print(cm)

Test Accuracy: 0.5712098009188361
Precision: 0.5686059006221679
Recall: 0.5712098009188361
F1-score: 0.5645641084581047
Confusion matrix:
[[  9  28  13]
 [  9 176 101]
 [  3 126 188]]


## LDA

In [282]:
lda=joblib.load('/content/lda.joblib')

In [283]:

y_pred = lda.predict(scaled_data)
ytest=y

print("Test Accuracy:", accuracy_score(ytest, y_pred))
cm = confusion_matrix(ytest, y_pred)
# precision = precision_score(ytest, y_pred, average='weighted')
# recall = recall_score(ytest, y_pred, average='weighted')
# f1 = f1_score(ytest, y_pred, average='weighted')
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion matrix:")
print(cm)

Test Accuracy: 0.5819295558958653
Precision: 0.5686059006221679
Recall: 0.5712098009188361
F1-score: 0.5645641084581047
Confusion matrix:
[[  7  30  13]
 [ 13 167 106]
 [  6 105 206]]


## Random forest

In [284]:
rf=joblib.load('/content/rf.joblib')

In [285]:
y_pred=rf.predict(scaled_data)
print("Accuracy:", accuracy_score(ytest, y_pred))
cm = confusion_matrix(ytest, y_pred)
precision = precision_score(ytest, y_pred, average='weighted')
recall = recall_score(ytest, y_pred, average='weighted')
f1 = f1_score(ytest, y_pred, average='weighted')#macro 
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion matrix:")
print(cm)

Accuracy: 0.6278713629402757
Precision: 0.6370709486373626
Recall: 0.6278713629402757
F1-score: 0.6142246631910544
Confusion matrix:
[[  6  35   9]
 [  2 184 100]
 [  0  97 220]]


## KNN

In [286]:
knn=joblib.load('/content/knn.joblib')

In [287]:

y_pred = knn.predict(scaled_data)
ytest=y
print("Test Accuracy:", accuracy_score(ytest, y_pred))
cm = confusion_matrix(ytest, y_pred)
precision = precision_score(ytest, y_pred, average='weighted')
recall = recall_score(ytest, y_pred, average='weighted')
f1 = f1_score(ytest, y_pred, average='weighted')
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion matrix:")
print(cm)

Test Accuracy: 0.6018376722817764
Precision: 0.6311216676563428
Recall: 0.6018376722817764
F1-score: 0.5817047849207114
Confusion matrix:
[[  2  34  14]
 [  0 174 112]
 [  0 100 217]]


## NN 

In [288]:
model=joblib.load('/content/NN.joblib')

In [289]:

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score


# Make predictions on new data
y_pred = model.predict(scaled_data)

21/21 [==============================] - 0s 4ms/step


In [290]:
from sklearn.metrics import fbeta_score

# Convert the test target variable to one-hot encoding
y_test_one_hot = to_categorical(y)

# Make predictions on new data
y_pred = model.predict(scaled_data)

# Convert predictions to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate accuracy score
f2_score = fbeta_score(y_test_one_hot, to_categorical(y_pred_labels), beta=2,average='weighted')
print('Accuracy:', f2_score)

# Print classification report
# print('Classification report:', classification_report(y_test_one_hot, to_categorical(y_pred_labels)))





21/21 [==============================] - 0s 2ms/step
Accuracy: 0.5546993033373229


In [291]:
from sklearn.metrics import f1_score

# Convert the test target variable to one-hot encoding
y_test_one_hot = to_categorical(ytest)

# Make predictions on new data
y_pred = model.predict(scaled_data)

# Convert predictions to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate accuracy score
f1_score = f1_score(y_test_one_hot, to_categorical(y_pred_labels), average='weighted') #macro
print('f1 Accuracy:', f1_score)
gg = accuracy_score(y_test_one_hot, to_categorical(y_pred_labels))
print('Accuracy:', gg)
# Print classification report
# print('Classification report:', classification_report(y_test_one_hot, to_categorical(y_pred_labels)))





21/21 [==============================] - 0s 2ms/step
f1 Accuracy: 0.5531654122523405
Accuracy: 0.55895865237366


In [292]:
from sklearn.metrics import classification_report

# Convert y_pred to categorical labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Convert y_test_one_hot back to original labels
y_test_labels = np.argmax(y_test_one_hot, axis=1)

# Compute classification report and confusion matrix
print('classification_report :', classification_report(y_test_labels, y_pred_labels))
cm = confusion_matrix(y_test_labels, y_pred_labels)
print('last: ', cm)




classification_report :               precision    recall  f1-score   support

           0       0.33      0.18      0.23        50
           1       0.51      0.66      0.58       286
           2       0.65      0.53      0.58       317

    accuracy                           0.56       653
   macro avg       0.50      0.46      0.46       653
weighted avg       0.56      0.56      0.55       653

last:  [[  9  33   8]
 [ 14 188  84]
 [  4 145 168]]


# PCA

In [293]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=2)
# xtrain_pca = pca.fit_transform(xtrain)

# df = pd.DataFrame({'PC1': xtrain_pca[:, 0], 'PC2': xtrain_pca[:, 1], 'target': ytrain})
# df



# # ------------------------------------------------------------
# # Import libraries
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn import datasets
# from sklearn.decomposition import PCA
# import pandas as pd
# from sklearn.preprocessing import StandardScaler

# # Load iris dataset
# iris = datasets.load_iris()
# X = iris.data
# y = iris.target

# # Scale the data to have zero mean and unit variance
# scaler = StandardScaler()
# scaler.fit(X)
# X_scaled = scaler.transform(X)

# # Apply PCA to reduce the dimensionality
# pca = PCA(n_components=8)
# X_pca = pca.fit_transform(X_scaled)

# # Select the components with the largest eigenvalues that explain most of the variance
# # You can use a threshold (e.g. 0.9) or a fixed number of components (e.g. 2)
# # Here we use a threshold of 0.9
# cum_var = np.cumsum(pca.explained_variance_ratio_)
# n_components = np.where(cum_var > 0.9)[0][0] + 1
# print(f"Number of components: {n_components}")
# X_selected = X_pca[:, :n_components]